In [8]:
def extract_packet_data(packet_data):
    # Ensure the packet data is at least 20 bytes (IPv4 header length)
    if (len(packet_data) < 40 ):  # Since the packet data is in hexadecimal, each byte is represented by 2 characters.
        raise ValueError("Invalid packet data: IPv4 header length should be at least 20 bytes.")

    # Extract the first byte and interpret it as IPv4 version and header length
    first_byte = int(packet_data[0:2], 16)
    ip_version = first_byte >> 4
    header_length = (first_byte & 0x0F) * 4  # Header length in bytes

    # Ensure it's a valid IPv4 packet
    if ip_version != 4 or header_length < 20:
        raise ValueError("Invalid IPv4 packet.")

    # Extract the total length of the packet (including the header)
    packet_length_hex = packet_data[4:8]
    packet_length = int(packet_length_hex, 16)

    # Ensure the packet data length matches the indicated packet length
    if (len(packet_data) != packet_length * 2):  # *2 because each byte is represented by 2 characters in hexadecimal.
        raise ValueError("Packet length mismatch.")
    
    # extract protocol
    protocol = int(packet_data[18:20], 16)

    # Extract source and destination IP addresses
    source_ip_hex = packet_data[24:32]
    destination_ip_hex = packet_data[32:40]

    # Convert the hexadecimal IP addresses to dot-decimal notation
    source_ip = ".".join(str(int(source_ip_hex[i : i + 2], 16)) for i in range(0, 8, 2))
    destination_ip = ".".join(str(int(destination_ip_hex[i : i + 2], 16)) for i in range(0, 8, 2))

    return {
        "version": ip_version,
        "header_length": header_length,
        "total_length": packet_length,
        "protocol": protocol,
        "source_ip": source_ip,
        "destination_ip": destination_ip,
    }

In [11]:
def parse_packet_data(packet_data):
    # Convert the hexadecimal data to bytes
    packet_bytes = bytes.fromhex(packet_data)

    # Extract the fields from the IPv4 header
    version = packet_bytes[0] >> 4
    header_length = (packet_bytes[0] & 0x0F) * 4
    total_length = int.from_bytes(packet_bytes[2:4], byteorder='big')
    identification = int.from_bytes(packet_bytes[4:6], byteorder='big')
    flags_fragment_offset = int.from_bytes(packet_bytes[6:8], byteorder='big')
    ttl = packet_bytes[8]
    protocol = packet_bytes[9]
    header_checksum = packet_bytes[10:12].hex()
    source_ip = ".".join(str(byte) for byte in packet_bytes[12:16])
    destination_ip = ".".join(str(byte) for byte in packet_bytes[16:20])

    payload = packet_bytes[header_length:]

    try:
        payload = payload.decode('utf-8')
    except UnicodeDecodeError:
        payload = payload

    return {
        "Version": version,
        "Header Length": header_length,
        "Total Length": total_length,
        "Identification": identification,
        "Flags and Fragment Offset": flags_fragment_offset,
        "Time to Live (TTL)": ttl,
        "Protocol": protocol,
        "Header Checksum": header_checksum,
        "Source IP Address": source_ip,
        "Destination IP Address": destination_ip,
        "Payload": payload
    }

In [12]:
with open("packets.txt", "r") as file:
    lines = file.readlines()

packets = {}
current_packet = None

# read line by line
for line in lines:
    line = line.strip()

    if line.isdigit():
        current_packet = int(line)
        packets[current_packet] = []
    else:
        packets[current_packet].append(line.split(":")[1].strip().replace(" ", ""))

# loop over packet in packets
for packet_number, packet_data in packets.items():
    packet_number = int(packet_number)  # First line is the packet number
    packet_data = "".join(packet_data)  # Combine packet data

    # Extract data from the packet
    # extracted_data = extract_packet_data(packet_data)
    extracted_data = parse_packet_data(packet_data)
    print(extracted_data["Payload"])

    # print the packet number
    print(packet_number, "\t", packet_data)

    # Display the extracted data
    for key, value in extracted_data.items():
        print(f"{key}: {value}")

b'\x08\x00\xd0\xb6\x00\xd2\x00\x03d\xb1\xd8\x9e\x00\x00\x00\x00\x00\x00*Q\x00\x00\x00\x00\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f !"#$%&\'()*+,-./01234567'
1 	 45000054005500004001d1a9010203048e3a166b0800d0b600d2000364b1d89e0000000000002a5100000000101112131415161718191a1b1c1d1e1f202122232425262728292a2b2c2d2e2f3031323334353637
Version: 4
Header Length: 20
Total Length: 84
Identification: 85
Flags and Fragment Offset: 0
Time to Live (TTL): 64
Protocol: 1
Header Checksum: d1a9
Source IP Address: 1.2.3.4
Destination IP Address: 142.58.22.107
Payload: b'\x08\x00\xd0\xb6\x00\xd2\x00\x03d\xb1\xd8\x9e\x00\x00\x00\x00\x00\x00*Q\x00\x00\x00\x00\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f !"#$%&\'()*+,-./01234567'
b'\x08\x00Sk\x03]\x00\x02d\xb1\xd8\x9e\x00\x00\x00\x00\x00\x00\xa5\x12\x00\x00\x00\x00\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f !"#$%&\'()*+,-./01234567'
2 	 4500005401211fff4001b0de010203048e3a166b0800536b035d000264b1d

In [8]:
import ipaddress

def is_ping_of_death(packet_data):
    # Check if the packet data represents a Ping of Death attack
    # Ping of Death packets have a length exceeding the maximum IP packet size (65535 bytes)
    packet_length = int(packet_data[4:8], 16)
    return packet_length > 65535

def is_smurf_attack(packet_data, subnet):
    # Check if the packet data represents a Smurf attack
    # Smurf attacks involve ICMP echo requests sent to broadcast addresses in the specified subnet

    # Define the subnet to be allowed (142.58.22.0/24)
    allowed_subnet = ipaddress.IPv4Network("142.58.22.0/24")

    destination_ip = packet_data[24:32]
    dest_ip_address = ipaddress.IPv4Address(bytes.fromhex(destination_ip))

    print(dest_ip_address)

    # Check if both source and destination IPs belong to the allowed subnet
    if (dest_ip_address in allowed_subnet):
        return True  # Valid packet
    else:
        return False  # Malicious packet


In [9]:
packet_data = "45000054005500004001d1a9010203048e3a166b0800d0b600d2000364b1d89e0000000000002a5100000000101112131415161718191a1b1c1d1e1f202122232425262728292a2b2c2d2e2f3031323334353637"
subnet = "142.58.22.0/24"

if is_ping_of_death(packet_data):
    print("Ping of Death Attack Detected!")
elif is_smurf_attack(packet_data, subnet):
    print("Smurf Attack Detected!")
else:
    print("No Ping of Death or Smurf Attack Detected.")

1.2.3.4
No Ping of Death or Smurf Attack Detected.


In [10]:
def parse_packet(packet_data, fields):
    parsed_data = {}
    for field_name, offset, size in fields:
        start_index = offset * 2  # Convert offset to hexadecimal index
        end_index = start_index + size * 2
        field_data = packet_data[start_index:end_index]
        parsed_data[field_name] = field_data
    return parsed_data

packet_data = "45000054005500004001d1a9010203048e3a166b0800d0b600d2000364b1d89e0000000000002a5100000000101112131415161718191a1b1c1d1e1f202122232425262728292a2b2c2d2e2f3031323334353637"

fields_to_extract = [
    ("Version", 0, 1),             # Offset: 0, Size: 1 byte
    ("Total Length", 4, 4),        # Offset: 4, Size: 4 bytes
    ("Protocol", 9, 1),            # Offset: 9, Size: 1 byte
    ("Source IP", 12, 8),          # Offset: 12, Size: 8 bytes (IPv4 address)
    ("Destination IP", 20, 8),     # Offset: 20, Size: 8 bytes (IPv4 address)
    ("Source Port", 36, 4),        # Offset: 36, Size: 4 bytes (TCP/UDP port)
    ("Destination Port", 40, 4),   # Offset: 40, Size: 4 bytes (TCP/UDP port)
    ("Payload", 52, -1),           # Offset: 52, Extract all remaining bytes as payload
]

parsed_data = parse_packet(packet_data, fields_to_extract)

# Print the parsed fields
for field_name, field_value in parsed_data.items():
    print(f"{field_name}: {field_value}")

Version: 45
Total Length: 00550000
Protocol: 01
Source IP: 010203048e3a166b
Destination IP: 0800d0b600d20003
Source Port: 00002a51
Destination Port: 00000000
Payload: 


In [17]:
def is_ping_of_death(packet_data):
    # Convert the hexadecimal packet data to bytes
    packet_bytes = bytes.fromhex(packet_data)

    # Extract the IP datagram size from the packet
    ip_datagram_size = (packet_bytes[2] << 8) + packet_bytes[3]

    print(ip_datagram_size)

    # Calculate the maximum allowed IP datagram size
    max_datagram_size = 65535

    # Check if the packet's IP datagram size is within the allowed range
    if ip_datagram_size > max_datagram_size:
        return True
    
    return False


with open("packets.txt", "r") as file:
    lines = file.readlines()

packets = {}
current_packet = None

# read line by line
for line in lines:
    line = line.strip()

    if line.isdigit():
        current_packet = int(line)
        packets[current_packet] = []
    else:
        packets[current_packet].append(line.split(":")[1].strip().replace(" ", ""))

# loop over packet in packets
for packet_number, packet_data in packets.items():
    packet_number = int(packet_number)  # First line is the packet number
    packet_data = "".join(packet_data)  # Combine packet data

    print(packet_data)

    if is_ping_of_death(packet_data):
        print(packet_number, "Ping of Death attack detected!")
    else:
        print(packet_number, "No Ping of Death attack detected.")


45000054005500004001d1a9010203048e3a166b0800d0b600d2000364b1d89e0000000000002a5100000000101112131415161718191a1b1c1d1e1f202122232425262728292a2b2c2d2e2f3031323334353637
84
1 No Ping of Death attack detected.
4500005401211fff4001b0de010203048e3a166b0800536b035d000264b1d89e000000000000a51200000000101112131415161718191a1b1c1d1e1f202122232425262728292a2b2c2d2e2f3031323334353637
84
2 No Ping of Death attack detected.
4500002801f600004006d02f8e3a166b01020304a95f01bb3f4a3d130000000050020fc800000000
40
3 No Ping of Death attack detected.
45000028033000004006cef5010203048e3a166b01bba95f21de68313f4a3d1450120ba400000000
40
4 No Ping of Death attack detected.
45000028010100004006d1248e3a166b01020304a95f01bb3f4a3d1421de683250100afa00000000
40
5 No Ping of Death attack detected.
4500002803aa00004006ce7b010203048e3a166b01bba95f21de68323f4a3d1450110c7600000000
40
6 No Ping of Death attack detected.
4500002803de00004006ce47010203048e3a166b01bba95f21de68323f4a3d145014137e00000000
40
7 No Ping of Death a

In [38]:
import struct

data = bytes.fromhex('45000054005500004001d1a9010203048e3a166b0800d0b600d2000364b1d89e0000000000002a5100000000101112131415161718191a1b1c1d1e1f202122232425262728292a2b2c2d2e2f3031323334353637')

_, _, source_port, dest_port, _, _, flags = struct.unpack('!HH16xH', data[20:40])
print(source_port, dest_port, flags)


error: unpack requires a buffer of 22 bytes

In [20]:
def is_ping_of_death(packet_data):
    # Assuming packet_data is a hexadecimal string representing the packet contents

    # Convert the hexadecimal string to bytes
    packet_bytes = bytes.fromhex(packet_data)

    # Parse the IP header to get the offset value
    offset = ((packet_bytes[6] & 0x1F) << 8) + packet_bytes[7]

    print(offset)

    # Check if the offset exceeds the maximum allowed value (65528 bytes)
    if offset > 65528:
        return True
    else:
        return False

# Example packet data
packet_data = "4500005401211fff4001b0de010203048e3a166b0800536b035d000264b1d89e000000000000a51200000000101112131415161718191a1b1c1d1e1f202122232425262728292a2b2c2d2e2f3031323334353637"

if is_ping_of_death(packet_data):
    print("Ping of Death attack detected!")
else:
    print("No Ping of Death attack detected.")


2
No Ping of Death attack detected.


In [26]:
packet_data = "4500005401211fff4001b0de010203048e3a166b0800536b035d000264b1d89e000000000000a51200000000101112131415161718191a1b1c1d1e1f202122232425262728292a2b2c2d2e2f3031323334353637"

# Extract payload data from the packet
payload_data = packet_data[40:]

# Convert the payload data to ASCII
payload = "".join(chr(int(payload_data[i : i + 2], 16)) for i in range(0, len(payload_data), 2))

# Print the payload
print(payload)

# get payload offset size (in bytes)
payload_offset = int(packet_data[24:25], 16) * 4

print(payload_offset)

# get payload size (in bytes)
payload_size = int(packet_data[40:42], 16) * 2

print(payload_size)

# get payload data
payload_data = packet_data[payload_offset:payload_offset + payload_size]

print(payload_data)

 Sk] d±Ø      ¥     !"#$%&'()*+,-./01234567
0
16
4500005401211fff


In [27]:
def calculate_ipv4_packet_size(packet_data):
    # Assuming packet_data is a hexadecimal string representing the packet contents

    # Convert the hexadecimal string to bytes
    packet_bytes = bytes.fromhex(packet_data)

    # Extract the Total Length field from the IPv4 header (bytes 2 and 3)
    total_length = (packet_bytes[2] << 8) + packet_bytes[3]

    # Calculate the size of the entire packet (IPv4 header + payload)
    packet_size = total_length

    return packet_size

# Example packet data
packet_data = "45000054005500004001d1a9010203048e3a166b0800d0b600d2000364b1d89e0000000000002a5100000000101112131415161718191a1b1c1d1e1f202122232425262728292a2b2c2d2e2f3031323334353637"

ipv4_packet_size = calculate_ipv4_packet_size(packet_data)
print("Size of the correctly-formed IPv4 packet (header + payload):", ipv4_packet_size, "bytes")


Size of the correctly-formed IPv4 packet (header + payload): 84 bytes


In [36]:
def extract_max_offset(packet_data):
    # Extracting the fragment offset and flags field from the packet data
    fragment_offset_flags = int(packet_data[12:16], 16)
    
    # Extracting the fragment offset value (first 13 bits)
    fragment_offset = fragment_offset_flags & 0x1FFF
    
    return fragment_offset

def is_ping_of_death(packet_data):
    # Extracting the total length of the packet from the packet data
    total_length = int(packet_data[4:8], 16)

    print("Total length: " + str(total_length))
    
    # Extracting the maximum offset value from the packet data
    fragment_offset = extract_max_offset(packet_data)

    print("Fragment offset: " + str(fragment_offset))
    
    # Calculating the value to compare with the threshold (65535)
    comparison_value = total_length + (fragment_offset * 8)

    print("Comparison value: " + str(comparison_value))
    
    # Checking if the condition for Ping of Death attack is met
    return comparison_value > 65535

# Example packet data with a large offset
packet_data = "4500005401211fff4001b0de010203048e3a166b0800536b035d000264b1d89e000000000000a51200000000101112131415161718191a1b1c1d1e1f202122232425262728292a2b2c2d2e2f3031323334353637"

if is_ping_of_death(packet_data):
    print("Ping of Death attack detected!")
else:
    print("No Ping of Death attack detected.")


Total length: 84
Fragment offset: 8191
Comparison value: 65612
Ping of Death attack detected!


In [39]:
def extract_packet_data(packet_data):
    # Ensure the packet data is at least 20 bytes (IPv4 header length)
    if (len(packet_data) < 40 ):  # Since the packet data is in hexadecimal, each byte is represented by 2 characters.
        raise ValueError("Invalid packet data: IPv4 header length should be at least 20 bytes.")

    # Extract the first byte and interpret it as IPv4 version and header length
    first_byte = int(packet_data[0:2], 16)
    ip_version = first_byte >> 4
    header_length = (first_byte & 0x0F) * 4  # Header length in bytes

    # Ensure it's a valid IPv4 packet
    if ip_version != 4 or header_length < 20:
        raise ValueError("Invalid IPv4 packet.")

    # Extract the total length of the packet (including the header)
    packet_length_hex = packet_data[4:8]
    packet_length = int(packet_length_hex, 16)

    # Ensure the packet data length matches the indicated packet length
    if (len(packet_data) != packet_length * 2):  # *2 because each byte is represented by 2 characters in hexadecimal.
        raise ValueError("Packet length mismatch.")
    
    # extract protocol
    protocol = int(packet_data[18:20], 16)

    # Extract source and destination IP addresses
    source_ip_hex = packet_data[24:32]
    destination_ip_hex = packet_data[32:40]

    # Convert the hexadecimal IP addresses to dot-decimal notation
    source_ip = ".".join(str(int(source_ip_hex[i : i + 2], 16)) for i in range(0, 8, 2))
    destination_ip = ".".join(str(int(destination_ip_hex[i : i + 2], 16)) for i in range(0, 8, 2))

    # Check if the protocol is TCP (6)
    if protocol == 6:
        # Extract source and destination ports
        source_port_hex = packet_data[40:44]
        destination_port_hex = packet_data[44:48]
        source_port = int(source_port_hex, 16)
        destination_port = int(destination_port_hex, 16)

        # Extract TCP flags
        tcp_flags_hex = packet_data[48:50]
        tcp_flags = int(tcp_flags_hex, 16)

        return {
            "version": ip_version,
            "header_length": header_length,
            "total_length": packet_length,
            "protocol": protocol,
            "source_ip": source_ip,
            "destination_ip": destination_ip,
            "source_port": source_port,
            "destination_port": destination_port,
            "tcp_flags": tcp_flags,
        }
    else:
        return {
            "version": ip_version,
            "header_length": header_length,
            "total_length": packet_length,
            "protocol": protocol,
            "source_ip": source_ip,
            "destination_ip": destination_ip,
        }

In [42]:
extract_packet_data('45000028010100004006d1248e3a166b01020304a95f01bb3f4a3d1421de683250100afa00000000')

{'version': 4,
 'header_length': 20,
 'total_length': 40,
 'protocol': 6,
 'source_ip': '142.58.22.107',
 'destination_ip': '1.2.3.4',
 'source_port': 43359,
 'destination_port': 443,
 'tcp_flags': 63}

In [43]:
def is_syn_packet(packet_flags):
    return (packet_flags & 0x02) != 0

def is_ack_packet(packet_flags):
    return (packet_flags & 0x10) != 0

def is_rst_packet(packet_flags):
    return (packet_flags & 0x04) != 0

def is_fin_packet(packet_flags):
    return (packet_flags & 0x01) != 0

In [45]:
def extract_packet_data(packet_data):
    # Assuming packet_data is a hexadecimal string representing the packet
    # Example: "45000054005500004001d1a9010203048e3a166b0800d0b600d2000364b1d89e0000000000002a5100000000101112131415161718191a1b1c1d1e1f202122232425262728292a2b2c2d2e2f3031323334353637"

    # Convert packet_data hex string to bytes
    packet_bytes = bytes.fromhex(packet_data)

    # Extract relevant fields from the packet
    version = (packet_bytes[0] >> 4) & 0xF
    ihl = packet_bytes[0] & 0xF
    flags = packet_bytes[20] >> 13
    source_ip = ".".join(str(byte) for byte in packet_bytes[12:16])
    destination_ip = ".".join(str(byte) for byte in packet_bytes[16:20])
    
    # Assuming TCP header starts at byte 20
    tcp_header = packet_bytes[ihl * 4:ihl * 4 + 20]
    source_port = (tcp_header[0] << 8) + tcp_header[1]
    destination_port = (tcp_header[2] << 8) + tcp_header[3]
    tcp_flags = tcp_header[13]
    
    # Create and return the extracted data dictionary
    extracted_data = {
        "version": version,
        "flags": tcp_flags,
        "source_ip": source_ip,
        "destination_ip": destination_ip,
        "source_port": source_port,
        "destination_port": destination_port,
    }
    
    return extracted_data


In [47]:
extract_packet_data('45000028010100004006d1248e3a166b01020304a95f01bb3f4a3d1421de683250100afa00000000')['flags']

{'version': 4,
 'flags': 16,
 'source_ip': '142.58.22.107',
 'destination_ip': '1.2.3.4',
 'source_port': 43359,
 'destination_port': 443}

In [53]:
is_fin_packet(extract_packet_data('45000028010100004006d1248e3a166b01020304a95f01bb3f4a3d1421de683250100afa00000000')['flags'])

False

In [48]:
def is_syn_packet(packet_flags):
    return (packet_flags & 0x02) != 0

def is_ack_packet(packet_flags):
    return (packet_flags & 0x10) != 0

def is_rst_packet(packet_flags):
    return (packet_flags & 0x04) != 0

def is_fin_packet(packet_flags):
    return (packet_flags & 0x01) != 0